In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
!pip install evaluate
!pip install seqeval

In [ ]:
%%capture
!pip install --upgrade transformers

In [ ]:
%%capture
!pip install matplotlib-venn
!pip install pydot
!pip install cartopy
import cartopy
import pydot

In [ ]:
%%capture
!pip install accelerate -U

In [ ]:
import torch
import pandas as pd
import os
import json
import re
import xml.etree.ElementTree as ET
from collections import Counter
from tqdm import tqdm
import nltk
import zipfile
import tarfile
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from google.colab import drive
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding
import json
import sqlite3
import csv
from tqdm import tqdm
import pandas as pd
import torch
from google.colab import drive
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding
import os
import pandas as pd
from lxml import etree
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ETree
from pathlib import Path
import glob
from tqdm import tqdm
from google.colab import drive
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda")
json_data = '/content/drive/MyDrive/davodi.json'
persian_df = pd.read_json(json_data)
persian_df = persian_df.drop(['_id', 'sentence', 'frame', 'lexicalUnit', 'status', 'issuer', 'is_active', 'createdAt', 'updatedAt', 'PId', 'lang', 'description', 'lexicalUnitHint', 'reviewer', 'lexicalUnitHelper', 'frameHelper', 'frameName', 'lexicalUnitName'], axis = 1)

In [ ]:
def framenet_tags(FN_tags):
    framenet_pattern = []
    for FN_tag in FN_tags:
        tag_type = FN_tag.get('tagType')
        if tag_type == 5:
            element = FN_tag.get('element', {})
            element_name = element.get('name', '')
            framenet_pattern.append(element_name)
        else:
            framenet_pattern.append('O')
    return framenet_pattern

In [ ]:
persian_df['frameNetTags'] = persian_df['frameNetTags'].apply(lambda FN_tags : framenet_tags(FN_tags))

In [ ]:
device = torch.device("cuda")
sentences_path = "/content/drive/MyDrive/sentences.zip"

In [ ]:
os.system("unzip sentences.zip")

2304

In [ ]:
zip_file_path = '/content/drive/My Drive/sentences.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/sentences')

In [ ]:
directory = "/content/drive/MyDrive/sentences"
dataframes = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filepath = os.path.join(directory, filename)
        enpb_df = pd.read_json(filepath, lines=True)
        dataframes.append(enpb_df)

In [ ]:
def read_json_files(root_dir):
  dataframes = []
  for root, dirs, files in os.walk(root_dir):
      for filename in files:
          if filename.endswith(".json"):
              filepath = os.path.join(root, filename)
              df = pd.read_json(filepath, lines=True)
              dataframes.append(df)
  return dataframes
dataframes = read_json_files("/content/drive/MyDrive/sentences")

In [ ]:
def prepare_token_classification_data(json_data):
    all_roles = []
    for key, value in json_data.items():
        predicate = value['predicate']
        roles = value['roles']
        verb_index = int(key)
        roles[verb_index] = "B-V"
        roles = ['O' if role == '_' else role for role in roles]
        all_roles.append(roles)
    return all_roles

In [ ]:
def create_token_label_dataframe_from_columns(df):
    sentence_ids = df["sentence_id"].tolist()
    words = df['words'].tolist()
    predictions = df['predictions'].tolist()
    df_prepared = pd.DataFrame(columns=["sentence_id", "words", "predictions"])
    for i, (sentence_id, word, prediction) in enumerate(zip(sentence_ids, words, predictions)):
        for label in prediction:
            df_prepared.loc[len(df_prepared)] = [sentence_id, word, label]
    return df_prepared

In [ ]:
enpb_df

,sentence_id,words,predictions
0,0,"[Sadly, it, was, a, legacy, scorned, by, its, ...","{'5': {'predicate': 'scorn-v.01', 'roles': ['_..."
1,1,"[Those, who, scorn, or, censure, women, 's, ta...","{'2': {'predicate': 'scorn-v.01', 'roles': ['A..."
2,2,"[I, am, a, woman, scorned, ,, she, thought, no...","{'4': {'predicate': 'scorn-v.01', 'roles': ['_..."
3,3,"[The, legend, recounted, how, seventy, transla...","{'2': {'predicate': 'recount-v.01', 'roles': [..."
4,4,"[Sitting, in, his, half, -, constructed, conse...","{'0': {'predicate': 'sit-v.01', 'roles': ['_',..."
...,...,...,...
9622,10215,"[Unable, to, continue, ,, she, gestured, to, E...","{'2': {'predicate': 'continue-v.01', 'roles': ..."
9623,10216,"[He, fastened, the, dress, with, clumsy, finge...","{'1': {'predicate': 'fasten-v.01', 'roles': ['..."
9624,10217,"[In, a, last, defiant, gesture, Diana, fastene...","{'6': {'predicate': 'fasten-v.01', 'roles': ['..."
9625,10218,"[He, fastens, the, bandage, with, sticky, tape...","{'1': {'predicate': 'fasten-v.01', 'roles': ['..."


In [ ]:
enpb_df['predictions'] = enpb_df['predictions'].apply(lambda json_data : prepare_token_classification_data(json_data))

In [ ]:
enpb_df = create_token_label_dataframe_from_columns(enpb_df)

In [ ]:
enpb_df = enpb_df.rename(columns={'predictions': 'propBankTags'})

In [ ]:
enpb_df['Sentence'] = enpb_df['words'].apply(lambda x: ' '.join(x))

In [ ]:
enpb_df

,sentence_id,words,propBankTags,Sentence
0,0,"[Sadly, it, was, a, legacy, scorned, by, its, ...","[O, O, O, O, A1, B-V, A0, O, O, O, O]",Sadly it was a legacy scorned by its putative ...
1,1,"[Those, who, scorn, or, censure, women, 's, ta...","[A0, R-A0, B-V, O, O, O, O, A1, O, O, O]",Those who scorn or censure women 's talk dimin...
2,1,"[Those, who, scorn, or, censure, women, 's, ta...","[A0, R-A0, O, O, B-V, O, O, A1, O, O, O]",Those who scorn or censure women 's talk dimin...
3,1,"[Those, who, scorn, or, censure, women, 's, ta...","[A0, O, O, O, O, O, O, O, B-V, A1, O]",Those who scorn or censure women 's talk dimin...
4,2,"[I, am, a, woman, scorned, ,, she, thought, no...","[O, O, O, A1, B-V, O, O, O, O, O, O, O, O, O, ...","I am a woman scorned , she thought now , in th..."
...,...,...,...,...
26445,10218,"[He, fastens, the, bandage, with, sticky, tape...","[A0, B-V, O, A1, A2, O, O, O]",He fastens the bandage with sticky tape .
26446,10219,"[The, six, relics, go, in, ,, and, ,, after, c...","[O, O, A1, B-V, AM-DIR, O, O, O, O, O, O, O, O...","The six relics go in , and , after carefully e..."
26447,10219,"[The, six, relics, go, in, ,, and, ,, after, c...","[O, O, O, O, O, O, O, O, O, AM-MNR, B-V, O, A1...","The six relics go in , and , after carefully e..."
26448,10219,"[The, six, relics, go, in, ,, and, ,, after, c...","[O, O, O, O, O, O, O, O, AM-TMP, O, O, O, O, O...","The six relics go in , and , after carefully e..."


In [ ]:
def process_label(label, all_words):
    start = label.get('start')
    end = label.get('end')
    if start is not None or end is not None:
        start = int(start)
        end = int(end)
        accumulated_chars = 0
        start_word_index = 0
        end_word_index = 0
        for i, word in enumerate(all_words):
            word_length = len(word)
            if accumulated_chars <= start:
                start_word_index = i
            if accumulated_chars >= end:
                end_word_index = i - 1
                break
            accumulated_chars += word_length + 1
        if end_word_index is None:
            end_word_index = len(all_words) - 1
        extracted_word = ' '.join(all_words[start_word_index:end_word_index + 1])
        return start, end, label.get('name'), extracted_word
    else:
        return None

In [ ]:
def annotate_sentence(df):
    sentence = df['Sentence']
    words = df['words']
    start = df['Start']
    end = df['End']
    annotation = df['Annotated Names']
    annotated_words = df['Extracted Words']
    result = []
    char_index = 0
    for word in words:
        label = 'O'
        word_start = char_index
        word_end = char_index + len(word) - 1
        if word in annotated_words.split():
            label = annotation
        result.append(label)
        char_index = word_end + 1
    return result

In [ ]:
xml_folder_path = '/content/drive/My Drive/framenetdata-1.7/fndata-1.7/lu'
xml_files = glob.glob(xml_folder_path + '/*.xml')
table = []
namespace = {'fn': 'http://framenet.icsi.berkeley.edu'}

In [ ]:
for filename in tqdm(xml_files):
    try:
        tree = ET.parse(filename)
        root = tree.getroot()
        for sentence in root.findall('.//fn:sentence', namespace):
            text_element = sentence.find('fn:text', namespace)
            if text_element is not None and text_element.text:
                all_words = text_element.text.split()
                annotation_sets = sentence.findall('.//fn:annotationSet[@status="MANUAL"]', namespace)
                for annotation_set in annotation_sets:
                    layers = annotation_set.findall('.//fn:layer[@name="FE"]', namespace)
                    for layer in layers:
                        labels = layer.findall('.//fn:label', namespace)
                        for label in labels:
                            processed_label = process_label(label, all_words)
                            if processed_label is not None:
                                table.append((text_element.text,) + processed_label)
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        print(f"XML file Name:" + str(filename))

 41%|████      | 1022/2483 [01:22<00:14, 102.40it/s]

Error parsing XML file: no element found: line 2, column 0
XML file Name:/content/drive/My Drive/framenetdata-1.7/fndata-1.7/lu/lu3973.xml


100%|██████████| 2483/2483 [01:39<00:00, 24.99it/s] 


In [ ]:
enfn_df = pd.DataFrame(table, columns=['Sentence', 'Start', 'End', 'Annotated Names', 'Extracted Words'])
enfn_df["words"] = enfn_df["Sentence"].apply(lambda sentence: [word.strip() for word in sentence.split(' ')])
enfn_df['frameNetTags'] = enfn_df.apply(annotate_sentence, axis=1)
enfn_df = enfn_df.drop(['Start', 'End', 'Annotated Names', 'Extracted Words'], axis = 1)
enfn_df

,Sentence,words,frameNetTags
0,` What makes you think Rainbow would entertain...,"[`, What, makes, you, think, Rainbow, would, e...","[O, O, O, Entity_2, O, O, O, O, O, O, O, O, O,..."
1,` What makes you think Rainbow would entertain...,"[`, What, makes, you, think, Rainbow, would, e...","[O, O, O, O, O, O, O, O, O, O, O, O, O, Entity..."
2,It aims to compete with the likes of BMW and A...,"[It, aims, to, compete, with, the, likes, of, ...","[O, O, O, O, O, O, O, Entity_2, Entity_2, Enti..."
3,It aims to compete with the likes of BMW and A...,"[It, aims, to, compete, with, the, likes, of, ...","[O, O, O, O, O, O, Entity_1, O, O, O, O, O, O,..."
4,They anathematised and expelled the likes of A...,"[They, anathematised, and, expelled, the, like...","[O, O, O, O, O, O, Entity_2, Entity_2, Entity_..."
...,...,...,...
59142,"During his four days in Iraq , Hall said he wa...","[During, his, four, days, in, Iraq, ,, Hall, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, Und..."
59143,"During his four days in Iraq , Hall said he wa...","[During, his, four, days, in, Iraq, ,, Hall, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
59144,"But the many whose lives he had ruined , whose...","[But, the, many, whose, lives, he, had, ruined...","[O, O, O, O, O, O, O, O, O, O, O, Undesirable_..."
59145,"But the many whose lives he had ruined , whose...","[But, the, many, whose, lives, he, had, ruined...","[O, O, O, O, O, O, O, O, O, O, Patient, Patien..."


In [ ]:
enpb_df

,sentence_id,words,propBankTags,Sentence
0,0,"[Sadly, it, was, a, legacy, scorned, by, its, ...","[O, O, O, O, A1, B-V, A0, O, O, O, O]",Sadly it was a legacy scorned by its putative ...
1,1,"[Those, who, scorn, or, censure, women, 's, ta...","[A0, R-A0, B-V, O, O, O, O, A1, O, O, O]",Those who scorn or censure women 's talk dimin...
2,1,"[Those, who, scorn, or, censure, women, 's, ta...","[A0, R-A0, O, O, B-V, O, O, A1, O, O, O]",Those who scorn or censure women 's talk dimin...
3,1,"[Those, who, scorn, or, censure, women, 's, ta...","[A0, O, O, O, O, O, O, O, B-V, A1, O]",Those who scorn or censure women 's talk dimin...
4,2,"[I, am, a, woman, scorned, ,, she, thought, no...","[O, O, O, A1, B-V, O, O, O, O, O, O, O, O, O, ...","I am a woman scorned , she thought now , in th..."
...,...,...,...,...
26445,10218,"[He, fastens, the, bandage, with, sticky, tape...","[A0, B-V, O, A1, A2, O, O, O]",He fastens the bandage with sticky tape .
26446,10219,"[The, six, relics, go, in, ,, and, ,, after, c...","[O, O, A1, B-V, AM-DIR, O, O, O, O, O, O, O, O...","The six relics go in , and , after carefully e..."
26447,10219,"[The, six, relics, go, in, ,, and, ,, after, c...","[O, O, O, O, O, O, O, O, O, AM-MNR, B-V, O, A1...","The six relics go in , and , after carefully e..."
26448,10219,"[The, six, relics, go, in, ,, and, ,, after, c...","[O, O, O, O, O, O, O, O, AM-TMP, O, O, O, O, O...","The six relics go in , and , after carefully e..."


In [ ]:
endf = pd.merge(enpb_df, enfn_df, on='Sentence', how='inner')

In [ ]:
endf

,sentence_id,words_x,propBankTags,Sentence,words_y,frameNetTags
0,346,"[The, second, ,, and, quite, different, variet...","[O, O, O, O, O, O, A2, O, O, B-V, A1, O, O, O,...","The second , and quite different variety of wr...","[The, second, ,, and, quite, different, variet...","[O, O, Action, O, O, O, O, Action, O, O, O, Ac..."
1,346,"[The, second, ,, and, quite, different, variet...","[O, O, O, O, O, O, O, O, O, O, B-V, O, A1, O, ...","The second , and quite different variety of wr...","[The, second, ,, and, quite, different, variet...","[O, O, Action, O, O, O, O, Action, O, O, O, Ac..."
2,347,"[The, offence, of, affray, can, be, committed,...","[O, A1, O, O, AM-MOD, O, B-V, AM-MNR, O, O, O,...",The offence of affray can be committed by usin...,"[The, offence, of, affray, can, be, committed,...","[O, O, O, O, O, O, O, O, O, O, O, O, Action, O..."
3,347,"[The, offence, of, affray, can, be, committed,...","[O, O, O, O, O, O, O, O, B-V, O, O, O, A1, O, ...",The offence of affray can be committed by usin...,"[The, offence, of, affray, can, be, committed,...","[O, O, O, O, O, O, O, O, O, O, O, O, Action, O..."
4,347,"[The, offence, of, affray, can, be, committed,...","[O, O, O, O, O, O, O, O, O, O, B-V, O, A1, O, ...",The offence of affray can be committed by usin...,"[The, offence, of, affray, can, be, committed,...","[O, O, O, O, O, O, O, O, O, O, O, O, Action, O..."
...,...,...,...,...,...,...
1139,9791,"[He, laments, trying, to, do, too, much, in, h...","[A0, O, B-V, A1, O, O, O, AM-TMP, O, O, O, O]",He laments trying to do too much in his first ...,"[He, laments, trying, to, do, too, much, in, h...","[O, O, Complaint, Complaint, Complaint, Compla..."
1140,9791,"[He, laments, trying, to, do, too, much, in, h...","[A0, O, O, O, B-V, O, A1, AM-TMP, O, O, O, O]",He laments trying to do too much in his first ...,"[He, laments, trying, to, do, too, much, in, h...","[Complainer, O, O, O, O, O, O, O, O, O, O, O]"
1141,9791,"[He, laments, trying, to, do, too, much, in, h...","[A0, O, O, O, B-V, O, A1, AM-TMP, O, O, O, O]",He laments trying to do too much in his first ...,"[He, laments, trying, to, do, too, much, in, h...","[O, O, Complaint, Complaint, Complaint, Compla..."
1142,9839,"[That, creature, must, have, been, travelling,...","[O, A0, AM-MOD, O, O, B-V, AM-MNR, O, O, O, O, O]",That creature must have been travelling at 60 ...,"[That, creature, must, have, been, travelling,...","[Theme, Theme, O, O, O, O, O, O, O, O, O, O]"


In [ ]:
train_data, temp_data = train_test_split(endf, test_size=0.2, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))
train_data.to_csv("train_set.csv", index=False)
validation_data.to_csv("validation_set.csv", index=False)
test_data.to_csv("test_set.csv", index=False)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
tokenized_input = tokenizer(endf['Sentence'][0].split(' '), is_split_into_words=True)
tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

['[CLS]',
 'The',
 'second',
 ',',
 'and',
 'quite',
 'different',
 'vari',
 'ety',
 'of',
 'wrong',
 'doing',
 'inv',
 'olves',
 'prov',
 'oking',
 'the',
 'use',
 'of',
 'un',
 'law',
 'ful',
 'viol',
 'ence',
 'by',
 'others',
 ',',
 'by',
 'the',
 'use',
 'of',
 'threat',
 's',
 ',',
 'ab',
 'use',
 'or',
 'ins',
 'ults',
 '.',
 '[SEP]']

In [ ]:
def create_role_id_dictionary():
    """Creates a dictionary that maps PropBank roles to numerical IDs."""
    #roles = set()
    frame_elements = set()
    for _, row in endf.iterrows():
        #prop_str = row['propBankTags']
        framenet_str = row['frameNetTags']
        #roles.update(prop_str)
        frame_elements.update(framenet_str)
    #role_id_dict = defaultdict(int)
    frame_id_dict = defaultdict(int)
    index = 1
    #for role in roles:
    #    role_id_dict[role] = index
    #    index += 1
    for frame_element in frame_elements:
        frame_id_dict[frame_element] = index
        index += 1
    return frame_id_dict, frame_elements

In [ ]:
frame2id, frame_elements = create_role_id_dictionary()
print("Frame2ID:" + str(frame2id))
print("Frame Elements:" + str(frame_elements))

Frame2ID:defaultdict(<class 'int'>, {'Evaluee': 1, 'Behavior': 2, 'Degree': 3, 'Place': 4, 'Count': 5, 'Event': 6, 'Object': 7, 'Explanation': 8, 'Action': 9, 'Particular_iteration': 10, 'Entity': 11, 'O': 12})
Frame Elements:{'Evaluee', 'Behavior', 'Degree', 'Place', 'Count', 'Event', 'Object', 'Explanation', 'Action', 'Particular_iteration', 'Entity', 'O'}


In [ ]:
endf['frameNetTags'] = endf['frameNetTags'].apply(lambda label : [frame2id[l] for l in label])

<ipython-input-39-41c7fcc7858b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  endf['frameNetTags'] = endf['frameNetTags'].apply(lambda label : [frame2id[l] for l in label])


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words_x"], truncation=True, padding=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"frameNetTags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(endf)
tokenized_data = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_data = tokenized_data.train_test_split(test_size=0.3)

In [ ]:
import evaluate
seqeval = evaluate.load("seqeval")

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import ModernBertForTokenClassification
model = ModernBertForTokenClassification.from_pretrained("answerdotai/ModernBERT-base", num_labels=len(frame2id)+1,ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForTokenClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label2id = dict(frame2id)

In [ ]:
id2label = dict((v,k) for k,v in label2id.items())

In [ ]:
label_list = list(frame2id.keys())
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import classification_report
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels_bin = list(np.concatenate(true_labels))
    pred_labels_bin = list(np.concatenate(true_predictions))
    print(true_labels_bin)
    print(pred_labels_bin)
    report = classification_report(true_labels_bin, pred_labels_bin)
    metrics = precision_recall_fscore_support(true_labels_bin, pred_labels_bin, average='weighted')
    accuracy = metrics[0]
    precision = metrics[1]
    recall = metrics[2]
    f1_score = metrics[3]
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1_score,
        "accuracy": accuracy
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="bert-token-classification",
    learning_rate=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    weight_decay=0.01,
    save_strategy="no",
    #use_cpu=True,
    #no_cuda=True,
    #load_best_model_at_end=True, # uncomment if you wanna save
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(model=model,args=training_args,data_collator=data_collator,train_dataset=tokenized_data['train']  ,eval_dataset=tokenized_data['test'],compute_metrics=compute_metrics,)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()